In [6]:
import ee
import geemap

ee.Authenticate()
ee.Initialize()

# Define the area of Washington State
wa_geometry = ee.FeatureCollection("TIGER/2018/States").filter(ee.Filter.eq('NAME', 'Washington')).geometry()

# Define Snoqualmie watershed
snoqualmie_geometry = ee.FeatureCollection("USGS/WBD/2017/HUC08").filter(ee.Filter.eq('name', 'Snoqualmie')).geometry()

# Define Landsat ImageCollection
landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA")

# Define time period
start_date = '2015-01-01'
end_date = '2020-12-31'

# Define start and end DOY
start_doy = 244
end_doy = 258

In [7]:
def calculate_indices(image):
    # Extract bands
    NIR = image.select('B5')
    RED = image.select('B4')
    GREEN = image.select('B3')
    BLUE = image.select('B2')
    SWIR1 = image.select('B6')

    # Calculate spectral indices
    NDVI = NIR.subtract(RED).divide(NIR.add(RED)).rename('NDVI')
    MNDWI = GREEN.subtract(SWIR1).divide(GREEN.add(SWIR1)).rename('MNDWI')
    
    # Calculate EVI (Landsat 8)
    EVI = image.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',
        {'NIR': NIR, 'RED': RED, 'BLUE': BLUE}
    ).rename('EVI')

    # Calculate NDYI
    NDYI = GREEN.subtract(BLUE).divide(GREEN.add(BLUE)).rename('NDYI')

    # Add bands to the image
    image = image.addBands([NDVI, MNDWI, EVI, NDYI])

    return image


In [8]:
# Filter Landsat ImageCollection
landsat_filtered = landsat.filterBounds(snoqualmie_geometry).filterDate(start_date, end_date)

# Filter by start_doy and end_doy
landsat_filtered = landsat_filtered.filter(ee.Filter.dayOfYear(start_doy, end_doy))

# Cloud masking function
def maskClouds(image):
    # Select the QA band
    QA = image.select('BQA')
    # Create a mask to exclude cloudy pixels
    cloud_mask = QA.bitwiseAnd(1 << 4).eq(0)  # Exclude clouds (bit 4)
    # Apply the mask to the image
    return image.updateMask(cloud_mask)

# Apply cloud masking function
landsat_filtered = landsat_filtered.map(maskClouds)

# Apply spectral indices functions to filtered Landsat ImageCollection
landsat_with_indices = landsat_filtered.map(calculate_indices)

In [10]:
#Reduce the ImageCollection to a single Image (using median)
composite = landsat_with_indices.reduce(ee.Reducer.median())

# Create a Map object
Map = geemap.Map(center=[47.5433, -121.8363], zoom=9)  # Centered on Snoqualmie Watershed

# Clip the Image to the Snoqualmie watershed and visualize indices
indices_to_visualize = ['NDVI_median', 'MNDWI_median', 'EVI_median', 'NDYI_median']

clipped_image = composite.clip(snoqualmie_geometry)
    
for index_name in indices_to_visualize:
    # Select index
    index_image = clipped_image.select(index_name)
        
    # Add index layer to the map
    Map.addLayer(index_image, {}, f'{index_name}_Snoqualmie')

# Display the map
Map.addLayer(snoqualmie_geometry, {}, 'Snoqualmie Watershed')
Map.centerObject(snoqualmie_geometry, 9)
Map

Map(center=[47.58728226482567, -121.6926530031262], controls=(WidgetControl(options=['position', 'transparent_…

In [5]:
## Define export parameters
export_params = {
    'image': clipped_image.select(indices_to_visualize),
    'description': f'spectral_indices_{watershed_id}',
    'scale': 30,  # Adjust scale as needed
    'region': ee.Geometry(watershed_geometry).bounds()  # Convert dictionary to Geometry and then get bounds
}

# Export as GeoTIFF
task = ee.batch.Export.image.toDrive(**export_params)
task.start()

print("Export initiated!")


Export initiated!
